In [ ]:
__nbid__ = '0020'
__author__ = 'Michael Balogh <mbalogh@uwaterloo.ca>, Robert Nikutta <robert.nikutta@noirlab.edu>'
__version__ = '20240606' # yyymmdd
__datasets__ = ['gogreen_dr1']
__keywords__ = ['gemini llp','tap','cluster','photometry','redshift','file service','spectra','catalogues']

# GOGREEN Data Release 1 data access at Astro Data Lab

*Authors:* Michael Balogh (GOGREEN) \<mbalogh@uwaterloo.ca\>, Robert Nikutta (Astro Data Lab) \<robert.nikutta@noirlab.edu\>

This notebook is adapted from the original version produced by the GOGREEN survey team. The original can be found in `Scripts/DR1_Notebook.ipynb`.

While the original notebook relies on local access to the data files and catalogues, this Data Lab modifed notebook loads the catalogs through a TAP service (using simple SQL queries), and accesses all needed data files (FITS files with 1-d and 2-d spectra, images, etc.) through the Data Lab file service for GOGREEN DR1.

# Table of contents
* [Disclaimer & attribution](#attribution)
* [Imports & Config](#imports)
* [Jupyter & Pandas primer](#primer)
* [File access at Data Lab](#fileaccess)
* [Reading in catalogs from TAP](#tap)
* [Merging tables](#merge)
* [Plots](#plots)
  * [Velocity dispersion vs redshift, dynamical mass vs redshfit](#plots-velocity)
  * [UVJ colours with membership](#plots-colors)
  * [Compare spectroscopic and photometric redshifts](#plots-compare-redshifts)
  * [Redshift success](#plots-redshift-success)
  * [Stellar mass and redshift distribution](#plots-mstar-zdistro)
  * [Completeness plots](#plots-completeness)
  * [Spectrum](#plots-spectrum)
  * [Redshift success as a function of spectral SNR](#plots-redshift-success-vs-snr)
  * [Redshift success Part II](#plots-redshift-success-2)
  * [S/N as a function of magnitude for different populations of galaxies](#plots-snr-vs-mag)
  * [Plot summary of object data](#plots-summary-object)

<a class="anchor" id="attribution"></a>
# Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.

<a class="anchor" id="imports"></a>
# Imports & Config

In [ ]:
# std lib
from io import BytesIO
import gzip

# 3rd party
import numpy as np
np.seterr(divide='ignore',invalid='ignore')

import pandas as pd
pd.set_option('display.max_columns', 500) # convenient for 

import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

from astropy.io import fits, ascii
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3) # define cosmology

# Data Lab
from dl import queryClient as qc, storeClient as sc

<a class="anchor" id="primer"></a>
# Jupyter & Pandas primer

**Brief introduction to ipython notebooks/jupyter**

See also: http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/
Short video: https://www.youtube.com/watch?v=jZ952vChhuI

Executing basics:
    - There are two modes, command mode and edit mode. 
        - If you have selected a cell and see a cursor, you are in edit mode (the cell will be outlined in green). 
        - If the cell is outlined in blue, you are in command mode. 
        - To toggle between modes, hit escape, or click on the margin of the cell.

    -  **************** Run a cell, in either mode: shift + enter ****************
        - Notice that the cell number will change. Jupyter doesn't care what order the cells are in, it cares what order you execute them in. Pay attention to where you have defined variables.
        
    - Command mode has some shortcuts for the keys:
        - x - cut. This is also delete, if you don't remember to paste. There are limited undo's under the Edit menu
        - c - copy
        - v - paste
        
    See tool bar for more options. Keyboard shortcuts listed in Help>Keyboard shortcuts (shortcut 'h')
    
Helpful tips:
    You can check the requirements for a package by typing (for example)
    > help(np.arange)
    or 
    > np.arange? # this makes a popout window
    
    you can also use the tab key to see available options 
    > np.arange(<tab>
    
There are also multiple types of cells. This cell is a 'Raw NBConvert' type cell, which is not executable.
    - code (y) - executable in python
    - markdown (m) - renders markdown type, usefule for headings,tables,equations
    - raw NBConvert (r) - non executable
    - headings (1-6) - markdown with quick heading setup
    
    - you may accidentally switch cell types while in command mode. You can check the cell type from the drop down menu in the tool bar.




**Brief introduction to pandas:**
See also: https://pandas.pydata.org/ 

https://pandas.pydata.org/pandas-docs/stable/10min.html

Pandas uses Series and DataFrame structures. It is much like astropy.table.Table, or numpy tables, but has convenient indexing which helps agaist misaligning columns/rows.

When retrieve elements in a column, 

    column = table['column_header'] 
        (alternatively, table.column_header, if column_header is not also a key word)
    this is a Series, which retains the indexing of the dataframe
    
    
    to convert to an array:
    column = table['column_header'].values
    
To access a specific row and column:

    object = table.loc[row_index,'column_header']
    
To access all indices in the dataframe:

    indices = table.index
    
Convenient functions:

    table.query('column_header > 0')
    returns a 'dataframe' slice, maintaining the index structure of the parent dataframe
    
    equivalent to, but cleaner than: table[table['column_header'] > 0]

<a class="anchor" id="fileaccess"></a>
# File access to GOGREEN DR1 data products at Data Lab

All GOGREEN DR1 data products, in their original form, reside at Data Lab in a so-called file service, or "public VOSpace". To explore the file service and its contents, use the Data Lab `storeClient` (imported as `sc`):

In [ ]:
print(sc.services('gogreen_dr1'))

To list the contents of the top-level directory, use the `storeClient.ls()` method:

In [ ]:
print(sc.ls('gogreen_dr1://',format='long'))

To list subdirectories, you can simply do:

In [ ]:
print(sc.ls('gogreen_dr1://SPECTROSCOPY/OneD',format='long'))

In the remainder of this notebook, we will need a few directories repeatedly. Let's define them for convenience:

In [ ]:
oneddir = 'gogreen_dr1://SPECTROSCOPY/OneD/'  # 1-d spectra
twoddir = 'gogreen_dr1://SPECTROSCOPY/TwoD/'  # 2-d spectra
imdir = 'gogreen_dr1://PHOTOMETRY/IMAGES/'    # photometry and images

<a class="anchor" id="tap"></a>
# Read in three main tables

GOGREEN Data Release 1 contains three main catalogues:

1. clusters -- Contains information about each of the 26 clusters
2. redhsift -- Is the redshift catalogue, with one entry for each spectrum
3. photo -- Is the merged photometric catalogue

All three catalogues are loaded in the Data Lab TAP service as queryable tables. You can interact with the tables using the Data Lab `queryClient` (imported as `qc`). Here, let's list some information about the database (also called "schema"): 

In [ ]:
print(qc.schema('gogreen_dr1'))

More information about each table:

In [ ]:
for table in ('clusters','photo','redshift'):
    print(qc.schema('gogreen_dr1.%s' % table))

Each table can be queried using either SQL (Postgres flavor) or ADQL.
For the purpose of this notebook we will query and load the entire tables.

First, the `clusters` table:

(The Data Lab queryClient `query` method can convert the query result to various formats on the fly,
e.g. to a Pandas data frame)

In [ ]:
cluster_table = qc.query('select * from gogreen_dr1.clusters',fmt='pandas')
print("Column names:", cluster_table.columns)
cluster_table.head(10)  # first 10 rows of the table

Next, the `photo` table:

In [ ]:
phot_table = qc.query('select * from gogreen_dr1.photo',fmt='pandas')
print("Column names:", phot_table.columns)
phot_table.head(10)  # first 10 rows of the table

And finally, the `redshift` table:

In [ ]:
redshift_table = qc.query('select * from gogreen_dr1.redshift',fmt='pandas')
print("Column names:", redshift_table.columns)
redshift_table.head(10)  # first 10 rows of the table

<a class="anchor" id="merge"></a>
# Merge tables

Let's now merge the `photo` and `redshift` tables, using the `specid` and `cluster` columns.
This will return a table that has photometric information (if available) for every object in the redshift catalogue.

In [ ]:
cols_to_use

In [ ]:
# this way avoids duplicate columns (ie dont need to specify suffixes)
merge_col = ['specid']
cols_to_use = phot_table.columns.difference(redshift_table.columns).tolist() + merge_col
matched_table = pd.merge(redshift_table, phot_table[cols_to_use], how='left', \
                         left_on=['specid'], right_on=merge_col )
merge_col = ['cluster']

# Here attach suffix _c to distinguish between galaxy values (Redshift) and cluster values (Redshift_c)
matched_table = pd.merge(matched_table, cluster_table, how='left', \
                         left_on=['cluster'], right_on=merge_col, suffixes=['','_c'] )
print("Column names:", matched_table.columns.values)
matched_table.head(20)  # show the first 20 rows of the matched table

<a class="anchor" id="plots"></a>
# Sample plots

<a class="anchor" id="plots-velocity"></a>
## Velocity dispersion vs redshift, dynamical mass vs redshfit

Generate two plots:  redshift and dynamical mass as a function of redshift for each cluster in the sample.  Similar to Figure 1 in the DR paper.

In [ ]:
zred, sigma, sigma_err, cluster_id = cluster_table[['redshift','vdisp','vdisp_err','cluster_id']].values.T

groups = np.where( (cluster_id == 13) | (cluster_id == 14) )
sparcs = np.where( ((cluster_id>3) & (cluster_id<13)) | (cluster_id>14) )
spt = np.where( (cluster_id<=3) )

fig, ax = plt.subplots(figsize=(8,8))

ax.errorbar( zred[groups], sigma[groups], sigma_err[groups], fmt='o', ms=15, label='COSMOS/SXDF', c='b')
ax.errorbar( zred[sparcs], sigma[sparcs], sigma_err[sparcs], fmt='x', ms=15, label='SpARCS', c='k')
ax.errorbar( zred[spt], sigma[spt], sigma_err[spt], fmt='*', ms=15, label='SPT', c='r')
ax.legend()
ax.set_xlim(0.8,1.5)
ax.set_ylim(0,1499)
ax.set_ylabel('Velocity Dispersion (km/s)')
ax.set_xlabel('Redshift')
fig.show()

(Saro_A,Saro_B,Saro_C) = (939,2.91,0.33)
hz = cosmo.H(zred)
h70 = hz.value / 100.
lSaro_Mass = 15. + Saro_B * np.log10( sigma / (Saro_A*h70**Saro_C) )
dM = Saro_B * sigma_err / sigma / (np.log(10))

fig, ax = plt.subplots(figsize=(8,8))
ax.errorbar(zred[groups], lSaro_Mass[groups], dM[groups], fmt='o', ms=15, label='COSMOS/SXDF', c='b')
ax.errorbar(zred[sparcs], lSaro_Mass[sparcs], dM[sparcs], fmt='x', ms=15, label='SpARCS', c='k')
ax.errorbar(zred[spt], lSaro_Mass[spt], dM[spt], fmt='*', ms=15, label='SPT', c='r')
ax.set_xlim(0.8,1.5)
ax.set_ylim(12.1,15.49)
ax.set_ylabel(r'log$_{10} (M_{\rm dyn}/M_\odot)$ ')
ax.set_xlabel('Redshift')
fig.show()

<a class="anchor" id="plots-colors"></a>
## UVJ colours with membership

Generate Figure 2:  UVJ diagram showing spectroscopic members and nonmembers

In [ ]:
bin_range = ([0,1.8],[0.2,2.2])
cmap = 'gray_r'

# use full photometric catalogue
selection = '(zphot>0.8) & (zphot<1.5) & (mstellar>10**9.5) & (mstellar<10**12)'
UminV,VminJ,photz,mass = phot_table.query(selection)[['uminv','vminj','zphot','mstellar']].values.T
lmass = np.log10(mass)

selection = '(redshift_quality>2) & (redshift>=0.8) & (redshift<=1.5)'
UminVspec,VminJspec,mem = matched_table.query(selection)[['uminv','vminj','member']].values.T

# Show in greyscale all galaxies with stellar mass between 9.5 and 12, and photo-z between 0.8 and 1.5
Nbins = 20
N,xedges,yedges = np.histogram2d( VminJ, UminV, bins=Nbins, range=bin_range)
N = N.T
xcenters = (xedges[:-1] + xedges[1:]) / 2
ycenters = (yedges[:-1] + yedges[1:]) / 2
X, Y = np.meshgrid(xedges, yedges)

fig,ax=plt.subplots(1,1,figsize=(13,13))

field = ax.imshow(N, interpolation='nearest', origin='lower',cmap=cmap,vmin=0,vmax=800,extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])

ax.scatter(VminJspec, UminVspec, c='yellow', edgecolor='k', alpha=1, s=25, label='Spectroscopic non-members')

ax.scatter(VminJspec[mem==1], UminVspec[mem==1], c='red', edgecolor='k', alpha=1, s=25, label='Cluster members')

ax.legend()
ax.set_ylim(0.2,2.2)
ax.set_xlim(0,1.8)
ax.set_xlabel("(V-J)")
ax.set_ylabel("(U-V)")
fig.show()

<a class="anchor" id="plots-compare-redshifts"></a>
##  Compare spectroscopic and photometric redshifts
Generate Figure 4:  Difference between specz and photoz 

In [ ]:
def zspeczphot(qflag1=3, qflag2=10):
    selection = '(zphot>0) & (redshift>=0.7) & (redshift<=1.5) & (redshift_quality>={}) & (redshift_quality<={})'.format(qflag1, qflag2)
    zspec, zphot, uminv, vminj = matched_table.query(selection)[['redshift','zphot','uminv','vminj']].values.T
    
    quiescent = np.where( (vminj<1.5) & (uminv>1.3) & (uminv>vminj*0.88+0.59))
    dz = (zspec-zphot)/(1.+zspec)
    outlier = np.where(np.abs(dz) > 0.15)
    good = np.where(np.abs(dz) <= 0.15)
    N = np.size(zspec)
    Noutlier = np.size(zspec[outlier])
    print ('Outliers: {:.4f}'.format(Noutlier*1./N))
    print ('rms:  {:.4f}'.format(np.std(dz[good])))
    
    fig,ax = plt.subplots(figsize=(8,5))
    ax.scatter(zspec, dz, c='k', marker='o')
    ax.scatter(zspec[quiescent], dz[quiescent], c='r')
    
    if (qflag1 == qflag2): label = 'Redshift Quality = {}'.format(qflag1)
    else: label = '{}<Redshift Quality<{}'.format(qflag1,qflag2)
    ax.text(0.05,0.05, label, transform=ax.transAxes, fontsize=20)

    ax.set_xlim(0.8,1.5)
    ax.set_ylim(-0.5,0.5)
    ax.set_ylabel(r'(z$_{\rm spec}$-z$_{\rm phot}$)/(1+z$_{\rm spec}$)')
    ax.set_xlabel('Spectroscopic Redshift')
    fig.show()
    
zspeczphot(qflag1=2, qflag2=2)
zspeczphot(qflag1=3, qflag2=3)
zspeczphot(qflag1=4, qflag2=4)

<a class="anchor" id="plots-redshift-success"></a>
## Redshift success
Similar to Figure 7 in the paper, showing the fraction of good redshifts as a function of magnitude and photometric redshift, for galaxies with S/N>1 at 8500A.  Note the paper used S/N measured at 8600A so is slightly different.  Further along in this notebook we will calculate S/N at other wavelengths and then recreate Fig 7 exactly.

In [ ]:
snrlim = 1.0
zp = np.array([(0,0.8),(0.8,1.5),(1.5,3)])
colours = np.array(['blue','green','k','red','magenta'])
lines = np.array(['dashed','solid','dotted'])
nbins = 10

gogreen_select = (matched_table['objclass']==1)  & (matched_table['snr_8500_var']>snrlim) & (matched_table['spec_flag']<2)
goodz = (matched_table['redshift_quality']>=3)
fig = plt.figure(figsize=(8,5))
ax = fig.add_subplot(1,1,1)
cindex = -1
mag = -2.5*np.log10(matched_table['z_tot'])+25
zphot = matched_table['zphot']
for (z1,z2) in zp:
    cindex = cindex+1
    zselect = (zphot>z1) & (zphot<=z2)
    zall,bins = np.histogram(mag[gogreen_select & zselect],bins=nbins,range=(21,25))
    zgood,bins = np.histogram(mag[gogreen_select & goodz & zselect],bins=nbins,range=(21,25))
    xmid = 0.5*(bins[1:]+bins[:-1])

    select = np.where(zall>5)
    plt.plot(xmid[select],zgood[select].astype('float')/zall[select],label=str(z1)+"<z<"+str(z2),c=colours[cindex],linestyle=lines[cindex])
    
plt.xlabel(r"Total z$\prime$ Magnitude (AB)")
plt.legend()
plt.ylabel("Fraction of good redshifts")
ax.axis([22,24.25,0.,1.0])

<a class="anchor" id="plots-mstar-zdistro"></a>
## Stellar mass and redshift distribution
Figure 9 in Data Release paper

In [ ]:
select = '(redshift_quality>{})'.format(2)
redshift,mass,UV,VJ=phot_table.query(select)[['zspec','mstellar','uminv','vminj']].values.T
lmass=np.log10(mass)
Quiescent=((VJ<1.5) & (UV>1.3) & (UV>VJ*0.88+0.59))
fig,ax=plt.subplots(figsize=(8,5))
ax.scatter(redshift,lmass,c='k',marker='o')
ax.scatter(redshift[Quiescent],lmass[Quiescent],c='r')
ax.set_xlim(0.8,1.5)
ax.set_ylim(8.5,11.99)
cm=cluster_table['redshift']*0+11.8
ax.scatter(cluster_table['redshift'],cm,marker='v',c='green',s=50)
ax.set_ylabel(r'Stellar Mass (M$_\odot$)')
ax.set_xlabel('Redshift')

<a class="anchor" id="plots-completeness"></a>
## Completeness plots
Figure 11 in Data Release paper

In [ ]:
# Define range of stellar masses and log(dr_phys) to plot
range=([9.5,11.2],[-1,0.2])
# Number of bins in each dimension
Nbins=(5,4)
photzrange=0.1
# To look at a single cluster by name
#clusters=np.array(['SpARCS1634'])
# To look at a range of clusters selected by cluster_id
#selection = '(cluster_id>={}) & (cluster_id<{}) '.format(1,2)
#clusters=cluster_table.query(selection)['cluster']
# To combine all clusters:
clusters=None
cmap='jet'
cnt=0

phot_table['lmstellar']=np.log10(phot_table['mstellar'])
if clusters is None: clusters=cluster_table['cluster']
for cluster in clusters:
    cselection = '(cluster=="{}")'.format(cluster)
    cluster_z,cluster_RA,cluster_DEC=cluster_table.query(cselection)[['redshift','ra_best','dec_best']].values.T
    clusterpos=SkyCoord(ra=cluster_RA*u.deg,dec=cluster_DEC*u.deg,frame='icrs')
    print (cluster,cluster_z)
    cnt+=1
    # This doesn't match what is in the paper.  Need l68 and u68.
    phot_table['dz_u']=phot_table['zphot_u68']-cluster_z
    phot_table['dz_l']=phot_table['zphot_l68']-cluster_z
    photselect='(lmstellar>{}) & (lmstellar<{}) & (dz_l<0) & (dz_u>0) & (star<{})'.format(range[0][0],range[0][1],1)
    specselect='(lmstellar>{}) & (lmstellar<{}) & (dz_l<0) & (dz_u>0) & (star<{}) & (redshift_quality>={})'.format(range[0][0],range[0][1],1,3)
    catpos=SkyCoord(ra=phot_table.query(photselect)['ra'].values.T*u.deg,dec=phot_table.query(photselect)['dec'].values.T*u.deg,frame='icrs') 
    UV=phot_table.query(photselect)['uminv'].values.T
    VJ=phot_table.query(photselect)['vminj'].values.T
    ZQ=phot_table.query(photselect)['redshift_quality'].values.T
    masses=phot_table.query(photselect)['lmstellar'].values.T

    dr=catpos.separation(clusterpos)
    DA=cosmo.angular_diameter_distance(cluster_z)
    dr_phys=dr.radian*DA
    dr_phys=np.log10(dr_phys.value+0.0001)

    withspec=(ZQ>=3)
    Quiescent=((VJ<1.5) & (UV>1.3) & (UV>VJ*0.88+0.59))
    # Bin by radius and stellar mass
    N,xedges,yedges=np.histogram2d(masses,dr_phys,bins=Nbins,range=range)
    Nq,xqedges,yqedges=np.histogram2d(masses[Quiescent],dr_phys[Quiescent],bins=Nbins,range=range)

    Ns,xedges,yedges=np.histogram2d(masses[withspec],dr_phys[withspec],bins=Nbins,range=range)
    Nqs,xqedges,yqedges=np.histogram2d(masses[withspec & Quiescent],dr_phys[withspec & Quiescent],bins=Nbins,range=range)

    # There's certainly a more elegant way to do this but I don't know what it is.
    if (cnt==1):
        Ntotal=N.T
        Nqtotal=Nq.T
        Nstotal=Ns.T
        Nqstotal=Nqs.T
    else:
        Ntotal+=N.T
        Nqtotal+=Nq.T
        Nstotal+=Ns.T
        Nqstotal+=Nqs.T
xcenters = (xedges[:-1] + xedges[1:]) / 2
ycenters = (yedges[:-1] + yedges[1:]) / 2
X, Y = np.meshgrid(xedges, yedges)
fig,(ax1,ax2,ax3) = plt.subplots(1,3,figsize=(13,4),sharey=True,sharex=True)
cbax=fig.add_axes([0.91,0.16,0.02,0.72])
fig.subplots_adjust(wspace=0.04,bottom=0.16)
#print (Ntotal,Nstotal)
#print (Nqtotal,Nqstotal)
H=Nstotal/Ntotal
Hq=Nqstotal/Nqtotal
HSF=(Nstotal-Nqstotal)/(Ntotal-Nqtotal)
all=ax1.imshow(H, interpolation='nearest', origin='lower',cmap=cmap,vmin=0,vmax=1,extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax1.set_aspect('auto')
ax1.set_title('All galaxy types')
Q=ax2.imshow(Hq, interpolation='nearest', origin='lower',cmap=cmap,vmin=0,vmax=1,extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax2.set_aspect('auto')
ax2.set_title('Quiescent')
SF=ax3.imshow(HSF, interpolation='nearest', origin='lower',cmap=cmap,vmin=0,vmax=1,extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
ax3.set_aspect('auto')
ax3.set_title('Star-forming')
ax1.set_ylabel(r"Log $\Delta$R (Mpc)",labelpad=15)
ax2.set_xlabel(r"Log Stellar Mass ($M_\odot$)")
fig.colorbar(all,cax=cbax)
fig.set_tight_layout(False)
fig.show()

## The following scripts require access to more than just the main catalogues

<a class="anchor" id="plots-spectrum"></a>
## Plot spectrum
This requires access to the 1D spectra, in SPECTROSCOPY/OneD

In [ ]:
# Here I'm going to query the dataframe to find an object, and eventually pull up its spectrum
obj = redshift_table.query('(cluster == "{}") & (extver == {})'.format('SpARCS0219', 1))
print(obj)

In [ ]:
obj

In [ ]:
def get_wavelength_from_hdu(hdr):
    """
    get_wavelength_from_hdu(hdr)

    :param hdu: Fits table header

    Reads 'CRVAL1' 'NAXIS1' and 'CD1_1' to compute wavelength coverage
    """
    return np.arange(hdr['CRVAL1'], hdr['CRVAL1']+hdr['NAXIS1']*hdr['CD1_1'], hdr['CD1_1'])

def get_spectrum(hdu, extver, units='fl', return_frame='observed', redshift=0., objclass=None, bounds=None):
    """
    get_spec(hdu, index, index_var, z=0., scale=1.e-15, bounds=None, units='Fl')

    :param hdu:       Fits table hdu object
    :param index:     Extension of science frame
    :param index_var: Extension of variance frame
    :param return_frame:  What frame to return the wavelength units in. If 'rest', a redshift is required.
    :param redshift:  Redshift of galaxy to convert to rest-frame, if redshift = 0 returns observed-frame
    :param units:     Units of output spectrum, case insensitive

    :type units:      string, "Fl" or "Jy"
    :returns:         wavelengths, the spectrum, and the variance

    Access spectrum from fits file, convert to specified units in rest frame

    Note: Input spectrum must be in units erg cm^-2 s^-1 A^-1
    Note: Header values must give wavelength in Angstroms
    """
    import astropy.units as u

    extver = int(extver)

    units = units.lower()
    assert units in ['fl','maggies','fnu'], 'Error, units must be "Fl" or "maggies"'
    return_frame = return_frame.lower()
    assert return_frame in ['rest','observed'], 'Error, return_frame must be either "rest" or "observed"'

    scale = hdu['SCI',extver].header['FLUXSCAL']
    spec = hdu['SCI',extver].data / scale
    var = hdu['VAR',extver].data / scale**2
    lam = get_wavelength_from_hdu(hdu['SCI',extver].header)

    if objclass==4:
        spec, var = fix_bad_gclass_spec(lam, spec, var, bounds, frame='observed', redshift=redshift)

    if return_frame == 'rest': # convert from observed to rest wavelength
        assert redshift >= 0, 'ERROR: redshift must be positive, is {}'.format(redshift)
        from astropy.cosmology import WMAP9 as cosmo

        dl = (cosmo.luminosity_distance(redshift).to(u.pc).value / 10.)**(-2)
        spec *= (1. + redshift) / dl
        var *= ((1. + redshift) / dl)**2
        lam /= (1. + redshift)

    if units == "maggies":
        convers = (3.34e4 * lam**2)/3631.
        spec *= convers
        var *= convers**2
    elif units == "fnu":
        convers = (3.34e4 * lam**2)
        spec *= convers
        var *= convers**2

    return lam, spec, var


In [ ]:
# dataframes often work better when you know its index. 
# Add '.index' at the end to return a 'list' of the indices that match your query
obj = redshift_table.query('(cluster=="{}") & (specid == {})'.format('SpARCS0219', 105000004))
obj_index = obj.index.values[0] # take the first index value

# Open the fits table and get the spectrum in the observed frame
# This code is written with a few options: 
#       return_frame = observed or rest (need a redshfit)
#       units = Fl or Maggies 

#print(sc.ls('gogreen_dr1://SPECTROSCOPY/OneD',format='long'))

fits_path = oneddir + 'SpARCS0219_final.fits'

#with fits.open(fits_path) as hdu:
with fits.open(BytesIO(sc.get(fits_path))) as hdu:
    lam, spec, var = get_spectrum(hdu, redshift_table.loc[obj_index,'extver'], return_frame='observed') # get observed frame spectra


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(16,4))
ax.step(lam, spec, color='k', where='mid', lw=0.5)
ax.fill_between(lam, spec+np.sqrt(var), spec-np.sqrt(var), alpha=1, color='limegreen', step='mid')
ax.set(xlabel='Observed wavelength', ylabel='Flux', yscale='linear', 
       xlim=[0.99*np.min(lam), np.max(lam)/0.99])
plt.show()

<a class="anchor" id="plots-redshift-success-vs-snr"></a>
## Redshift success as a function of spectral SNR
Fig 6 in Data Release paper.  Requires measuring the S/N at different wavelengths.
First, a routine to calculate the S/N at different wavelengths, and save it to a file.  We can either take the noise from the VAR array (SNR_VAR) or measure the RMS in the spectrum at the wavelength of interest (SNR_RMS)

In [ ]:
def recalcSNR(snr_central=np.arange(8000,9801,200),dwave=200):
    oldcluster = 'None'
    outfilename = 'SNR.txt'
    outfile = open(outfilename,"w+")
    outfile.write("#Cluster EXTVER SPECID ")
    for w in snr_central:
        outfile.write("SNR_"+str(w)+"_VAR SNR_"+str(w)+"_RMS ")
    outfile.write("\n")
    for i in np.arange(np.size(redshift_table['specid'])):
        if i % 500 == 0:
            print(i)
        cluster = redshift_table['cluster'][i]
        ext = redshift_table['extver'][i]
        specid = redshift_table['specid'][i]
        if cluster != oldcluster:
            try:
                MEF.close()
            except:
                pass
            oldcluster = cluster
            onedfile = oneddir + cluster + '_final.fits'
            MEF = fits.open(BytesIO(sc.get(onedfile)))  # getting raw bytes from the file service via sc.get()
            #MEF = fits.open(onedfile)
        signal = MEF['sci',ext].data
        variance = MEF['var',ext].data
        header = MEF['sci',ext].header
        crval1 = header['CRVAL1']
        crpix1 = header['CRPIX1']
        cd1_1 = header['CD1_1']
        outfile.write("%s %d %d " % (cluster,ext,specid))
        for w in snr_central:
            wave=np.array([w-dwave,w+dwave])
            pix=((wave-crval1)/cd1_1+crpix1).astype(int)
            snr_var=np.average(signal[pix[0]:pix[1]]/np.sqrt(variance[pix[0]:pix[1]]))        
            if ((np.std(signal[pix[0]:pix[1]]))>0):
                snr_rms=np.average(signal[pix[0]:pix[1]])/np.std(signal[pix[0]:pix[1]])
            else:
                snr_rms=0
            outfile.write("%f %f " % (snr_var,snr_rms))
        outfile.write("\n")
    outfile.close()

In [ ]:
recalcSNR()

<a class="anchor" id="plots-redshift-success-2"></a>
## Redshift success Part II
Now we can make Figure 7 in the paper, by restricting to galaxies with S/N>1 at 9600 A. 

In [ ]:
snrlim = 1.0
snrwave = 9600
zp = np.array([(0,0.8),(0.8,1.5),(1.5,3)])
colours = np.array(['blue','green','k','red','magenta'])
lines = np.array(['dashed','solid','dotted'])
nbins = 10

#snrfile=catdir+'SNR.txt'
snrfile = 'SNR.txt'
snrdata = ascii.read(snrfile)
snr_rms = snrdata['SNR_'+str(snrwave)+'_RMS']
snr_var = snrdata['SNR_'+str(snrwave)+'_VAR']
gogreen_select = (matched_table['objclass']==1)  & (snr_var>snrlim) & (matched_table['spec_flag']<2)
goodz = (matched_table['redshift_quality']>=3)
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1,1,1)
cindex = -1
mag = -2.5*np.log10(matched_table['z_tot'])+25
zphot = matched_table['zphot']
for (z1,z2) in zp:
    cindex = cindex+1
    zselect = (zphot>z1) & (zphot<=z2)
    zall,bins = np.histogram(mag[gogreen_select & zselect],bins=nbins,range=(21,25))
    zgood,bins = np.histogram(mag[gogreen_select & goodz & zselect],bins=nbins,range=(21,25))
    xmid = 0.5*(bins[1:]+bins[:-1])

    select = np.where(zall>5)
    plt.plot(xmid[select],zgood[select].astype('float')/zall[select],label=str(z1)+"<z<"+str(z2),c=colours[cindex],linestyle=lines[cindex])
    
plt.xlabel(r"Total z$\prime$ Magnitude (AB)")
plt.legend()
plt.ylabel("Fraction of good redshifts")
ax.axis([22,24.25,0.,1.0])

<a class="anchor" id="plots-snr-vs-mag"></a>
# S/N as a function of magnitude for different populations of galaxies
Fig 3 in the Data release paper

In [ ]:
snrwave = 9600
snrfile = 'SNR.txt'
snrdata = ascii.read(snrfile)
snr_rms = snrdata['SNR_'+str(snrwave)+'_RMS']
snr_var = snrdata['SNR_'+str(snrwave)+'_VAR']
zmag = -2.5*np.log10(matched_table['z_tot'])+25
flags = matched_table['spec_flag']
gogreen_select = (matched_table['objclass']==1) 
gclass_select = (matched_table['objclass']==4)
goodz = (matched_table['redshift_quality']>=3)

fig, ax = plt.subplots(figsize=(8,6))
ax.scatter(zmag[gclass_select & goodz],snr_rms[gclass_select & goodz],c='k',s=5,marker='s')
ax.scatter(zmag[gogreen_select],snr_rms[gogreen_select],c='g',marker='.',s=2)
ax.scatter(zmag[gogreen_select & goodz],snr_rms[gogreen_select & goodz],c='g',s=10)
snlim = (snr_rms>1.0)
x1 = zmag[goodz & snlim]
x2 = zmag[goodz]
m1 = 21
m2 = 24.5
ax.axis([m1,m2,0.1,15])
y1 = 9
y2 = (10.**(-(m2-m1)/2.5))*y1
plt.plot((m1,m2),(y1,y2))
ax.set_yscale('log')
ax.get_yaxis().set_major_formatter(plticker.ScalarFormatter())
plt.ylabel("S/N per pixel at "+str(int(snrwave))+r"$\mathrm{\AA}$")
plt.xlabel(r"Total z$\prime$ Magnitude (AB)")
fig.show()

In [ ]:
# Specify an array of wavelengths at which you want to measure S/N.  Must correspond to entries in SNR.txt, generated above.
snrwave = np.array([8400,9600])
nbins = 15
colours = np.array(['k','red','blue','green'])
lines = np.array(['solid','dotted','dashed'])
snrfile = 'SNR.txt'
snrdata = ascii.read(snrfile)
flags = matched_table['spec_flag']
VJ = matched_table['vminj']
UV = matched_table['uminv']
Quiescent = (VJ<1.5) & (UV>1.3) & (UV>VJ*0.88+0.59)
zphot = matched_table['zphot']
gogreen_select = (matched_table['objclass']==1) & (flags<2) 
goodz = (matched_table['redshift_quality']>=3)  
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(1,1,1)
cindex = -1
for snr in snrwave:
    cindex = cindex+1
    snr_var = snrdata['SNR_'+str(snr)+'_VAR']
    zall_var,bins = np.histogram(np.log10(snr_var[gogreen_select]),bins=nbins,range=(-1,1))
    zgood_var,bins = np.histogram(np.log10(snr_var[gogreen_select & goodz]),bins=nbins,range=(-1,1))
    xmid = 0.5*(bins[1:]+bins[:-1])
    ok = np.where(zall_var>10)
    plt.semilogx(10.**xmid[ok],zgood_var[ok].astype('float')/zall_var[ok],label="%d Å, All" % snr,c='k',linestyle=lines[cindex])
    zall_var,bins = np.histogram(np.log10(snr_var[gogreen_select & Quiescent]),bins=nbins,range=(-1,1))
    zgood_var,bins = np.histogram(np.log10(snr_var[gogreen_select & goodz & Quiescent]),bins=nbins,range=(-1,1))
    xmid = 0.5*(bins[1:]+bins[:-1])
    ok = np.where(zall_var>10)
    plt.semilogx(10.**xmid[ok],zgood_var[ok].astype('float')/zall_var[ok],label="%d Å, Quiescent" % snr,c='red',linestyle=lines[cindex])
plt.xlabel(r"S/N per pixel")
plt.legend()
ax.get_xaxis().set_major_formatter(plticker.ScalarFormatter())
plt.ylabel("Fraction of good redshifts")
plt.tick_params(which='major', length=8,top='on', bottom='on', left='on', right='on')
plt.tick_params(which='minor', length=5,top='on', bottom='on', left='on', right='on')
ax.axis([0.4,10.1,0.,1.0])
fig.show()

<a class="anchor" id="plots-summary-object"></a>
## Plot summary of object data
Here is another example of a routine that will plot the 1D and 2D spectrum, together with the I-band image and some other information, for a given object.  To plot the image it nees access to the PHOTOMETRY/IMAGES/ directory, but it should run without it (you just won't get the image)

In [ ]:
def markline(ax, name, wavelength, location, ll, y, y1, y2):
    # adds an annotation with vertical line and label
    doline = True
    assert location.lower() in ['below','above'], 'Error: "location" must be either "below" or "above"'
    try:
        range2 = np.where((ll>wavelength*0.995) & (ll<wavelength*1.005) & (y>0))
        s1 = np.min(y[range2])
        s2 = np.max(y[range2])
    except:
        doline = False
    if doline:
        gap = 0.25*(y2-y1)
        if location == 'below':
            textpos = s1-gap
        elif location == 'above':
            textpos = s2+gap
        ax.annotate(name, xy=(wavelength, s2), xytext=(wavelength,textpos), \
                    arrowprops=dict(facecolor='black', shrink=0.05, width=1, headwidth=0))

def plotspecinspect(table, obj_index,filter='I'):
    import matplotlib.patches as patches
    import matplotlib.transforms as transforms
    import matplotlib.ticker as plticker
    import matplotlib.gridspec as gridspec

    from astropy import wcs
    from astropy.convolution import convolve, Box1DKernel

    import os
    
    # make variables of file names
    cluster = table.loc[obj_index,'cluster']
    onedfile = oneddir + cluster + '_final.fits'
    twodfile = twoddir + cluster + '_twod.fits.gz'
    imname = table.loc[obj_index,'image_'+filter].rstrip()
    cdirname = cluster.replace('SPT','SPTCL-').replace('SpARCS','SpARCS-')
    image = imdir + cdirname + '/' + imname + '.fits'
    imageaccess=True
    
    if sc.stat(image) == {}:
        image = image + '.gz'
        
    if sc.stat(image) == {}:
        print ('No image file ',image)
        imageaccess=False
        
    # Get some key information for this object and assign to useful variable names
    # To use the position from the spectroscopy catalogue (image may not align precisely with overlaid slit position)
    #    cols = ['SPECID','EXTVER','Redshift','zphot','Redshift_Quality','RA(J2000)','DEC(J2000)','OBJClass','PA_deg','z_tot','SNR_8500_VAR']
    # To use position from the photometric catalogue
    cols = ['specid','extver','redshift','zphot','redshift_quality','ra','dec','objclass','pa_deg','z_tot','snr_8500_var']
    specid, ext, zspec, zphot, Quality, ra, dec, objclass, pa, z_tot, snr_target = table.loc[obj_index,cols].values.T
    radec = np.array([[ra,dec]])
    usephotoz = False # if redshfit quality below 3, use photmetric redshift instead
    
    zmag_target = -2.5 * np.log10(z_tot)+25
    
    if (Quality<3) & ~np.isnan(zphot): # check redshift quality, if <3 use phot (if exists)
        zred = zphot
        usephotoz = True
    else: 
        zred = zspec
    
    # Create figure
    fig = plt.figure(constrained_layout=True, figsize=(4*4, 4*3))
    gs = gridspec.GridSpec(ncols=4, nrows=3, figure=fig, height_ratios=[1,1,0.8])
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    ax3 = fig.add_subplot(gs[0, 2:])
    ax4 = fig.add_subplot(gs[1, :])
    ax5 = fig.add_subplot(gs[2, :])
    
    ##############################
    # 1 - Information panel
    info = '\n'.join([ 'SPECID = {}','ZMAG = {:.1f}','SNR = {:.2f}','OBJCLASS = {}','z = {:.3f}'])
    info = info.format(specid, zmag_target, snr_target, objclass, zred)
    if usephotoz: info += '\nUzing photz'
    else: info += '\nz_Q = {}'.format(Quality)
    
    ax1.axis('off')
    ax1.text(x=0, y=0, s=info, fontsize=20)
    
    
    ##############################
    # 2 - Postage stamp from preimage
    if imageaccess:
        # Open file
        if image.endswith('.fits'):
            imagedat = fits.open(BytesIO(sc.get(image)))
        elif image.endswith('.fits.gz'):
            imagedat = fits.open(gzip.GzipFile(fileobj=BytesIO(sc.get(image))))
        
        try: 
            im = imagedat[1].data
            im_header = imagedat[1].header
        except:
            im = imagedat[0].data
            im_header = imagedat[0].header
            
        imagedat.close()
        
        # from pixel scale, calculate CCD scale
        imagewcs = wcs.WCS(im_header)
        im_xscale = (np.abs(im_header['CD1_1'])+np.abs(im_header['CD1_2']))*3600
        im_yscale = (np.abs(im_header['CD2_1'])+np.abs(im_header['CD2_2']))*3600
        im = im-np.median(im)
        pixels = imagewcs.wcs_world2pix(radec,0)
        yccd, xccd = pixels[0][0], pixels[0][1]

        # Create postage stamp of galaxy
        defstampsize = 5 / im_xscale * np.array([1,1])
        ximage = [int(np.max([xccd-defstampsize[0]/2+.5,0])),int(np.min([xccd+defstampsize[0]/2+.5,np.shape(im)[0]]))]
        yimage = [int(np.max([yccd-defstampsize[1]/2+.5,0])),int(np.min([yccd+defstampsize[1]/2+.5,np.shape(im)[1]]))]
        centralstampsize = 1 / im_xscale * np.array([1,1])
        xcent = [int(np.max([xccd-centralstampsize[0]/2+.5,0])),int(np.min([xccd+centralstampsize[0]/2+.5,np.shape(im)[0]]))]
        ycent = [int(np.max([yccd-centralstampsize[1]/2+.5,0])),int(np.min([yccd+centralstampsize[1]/2+.5,np.shape(im)[1]]))]
        xstamp = xccd-ximage[0]
        ystamp = yccd-yimage[0]

        ax2.axis('off')
        try:
            im_sub = im[ximage[0]:ximage[1],yimage[0]:yimage[1]]
            minval = np.nanmin(im_sub)
            maxval = np.nanmax(im[xcent[0]:xcent[1],ycent[0]:ycent[1]])
            ax2.imshow(im_sub, vmax=0.99*maxval, vmin=minval, cmap=plt.get_cmap('gray'))
        except:
            pass
        # rectangle: xy=(), width, height
        rect = patches.Rectangle(
            (xstamp-1/2./im_xscale, ystamp-2.25/im_yscale),
            1/im_xscale,
            4.5/im_yscale,
            fill=False,
            edgecolor='red')
        obj = patches.Rectangle(
            (xstamp-1./2./im_xscale, ystamp-0.775/im_yscale),
            1/im_xscale,
            1.55/im_yscale,
            fill=False,
            edgecolor='yellow')
        t = transforms.Affine2D().rotate_deg_around(xstamp,ystamp,pa)
        rect.set_transform(t + ax2.transData)
        obj.set_transform(t + ax2.transData)
        ax2.add_patch(rect)
        ax2.add_patch(obj)
        ax2.scatter(xstamp,ystamp,marker = 'x')
        ax2.set_xlim(0,int(defstampsize[0]))
        ax2.set_ylim(0,int(defstampsize[1]))
    
    
    ##############################
    # 3 - SNR vs ZMAG
    ztots, snrs = matched_table.query('objclass==1')[['z_tot','snr_8500_var']].values.T
    zmags = -2.5*np.log10(ztots) + 25
    
    ax3.scatter(zmags, snrs, facecolor='none', edgecolor='k', s=50)
    ax3.scatter(zmag_target, snr_target, c='r', s=50)
    ax3.set_yscale('log')
    ax3.get_yaxis().set_major_formatter(plticker.ScalarFormatter())
    ax3.set_ylim(0.2,10)
    ax3.set_xlim(20,24.5)
    ax3.set_ylabel("S/N @ 850nm")
    ax3.set_xlabel("ZMAG")
    
    ##############################
    ##### 4 - OneD spectrum
    
    # open file
    with fits.open(BytesIO(sc.get(onedfile))) as onedspec:  # reading file from the file service
        mdf = onedspec['mdf',1].data
        spectrum = onedspec['sci',ext].data
        header_primary = onedspec[0].header
        
        hd = onedspec['sci',ext].header
        var = onedspec['var',ext].data
        unc = np.sqrt( var )
        
    # calculate wavelength array
    pix = np.arange(1,hd['naxis1']+1)
    ll = hd['crval1']+(pix-hd['crpix1'])*hd['cd1_1']
    
    smoothed = convolve(spectrum,Box1DKernel(5)) # smooth spectrum, just for plotting purposes

    ax4.fill_between(ll, spectrum-unc, spectrum+unc, color='yellow', step='mid' )
    ax4.step(ll, spectrum, c='gray', where='mid')
    ax4.step(ll, smoothed, c='k', where='mid')
    ax4.set_xlim(ll[0],ll[-1])
    y1 = np.nanmean(spectrum[:-200])-2.*np.nanstd(spectrum[:-200])
    y2 = np.nanmean(spectrum[:-200])+5.*np.nanstd(spectrum[:-200])

    ax4.set_ylim(y1,y2)
    ax4.set_xlabel("Observed frame wavelength (A)")

    # add markers of spectral features
    tax4 = ax4.twiny()
    llz = ll/(1.+zred)

    tax4.set_xlim(llz[0], llz[-1])
    tax4.set_xlabel("Rest frame wavelength (A)")
    y1 = np.max([1.e-3,np.min(smoothed)])
    y2 = 1.2*np.max(smoothed)
    
    for label,wl,placement in [['MgII',2800,'above'],
                               ['[OII]',3727,'below'],
                               ['H$\delta$',4102,'above'],
                               ['',4341,'above'],
                               ['H',3970,'above'],
                               ['',3889,'above'],
                               ['',3835,'above'],
                               ["",3797,'above'],
                               ['',3770,'above'],
                               ['CaK',3934,'above'],
                               ['G',4300,'above'],
                               ['FeII',2586,'above'],
                               ['',2600,'above'],
                               ['FeII',2374,'above'],
                               ['',2382.76,'above'],
                               ['FeII',2344,'above'],
                               ['MgI',2850,'above']]:
        
        markline(tax4, label,wl,placement, llz, smoothed, y1, y2)

    ##############################
    ##### 4 - TwoD spectrum sky-subtracted slid with trace
    
    # Open 2D spectrum if it exists
    twodexists = True
    # Reading raw bits from the file service via sc.get(),
    # then into a BytesIO object (behaves like a file in RAM),
    # then unzipping it with gzip,
    # and finally opening it with astropy.io.fits
    with fits.open(gzip.GzipFile(fileobj=BytesIO(sc.get(twodfile)))) as twodspec:

        try:
            twod = twodspec['sci',ext].data
        except:
            twodexists = False    

    if twodexists:
        maxval = 0.99*np.nanmax(np.abs(np.nanmedian(twod[2:8,950:1100],1)))
        if (maxval<= 0):
            maxval = 0.99*np.nanmax(np.abs(np.nanmedian(twod,1)))
        extent = [ll[0],ll[-1],1,20]
        ax5.imshow(twod[:,:], extent=extent, origin='lower', aspect=10, vmax=maxval, vmin=-maxval, cmap=plt.get_cmap('gray'))
        ax5.set_xlim(ll[0],ll[-1])
        ax5.set_xlabel("Observed frame wavelength (A)")
        

    fig.show()

Here's an example of how to use this to generate plots for objects selected based on their cluster, redshift quality and z magnitude. By default this i plotting the I-band image but you can change this by specifying a different filter suffix.

In [ ]:
cluster = 'SPT0205'
image_filter='i'
zmaglim = np.array([23.5, 23.0])
zfluxlim = 10.**(-0.4*(zmaglim-25.))

#objs = matched_table.query('(SPECID=="{}")'.format(211060584))
objs = matched_table.query('(cluster=="{}") & (redshift_quality == {}) & (z_tot >= {}) & (z_tot <= {})'.format(cluster, 4,zfluxlim[0],zfluxlim[1]))

for obj_index in objs.index.values:
    print ('Generating plot for specid ',matched_table.loc[obj_index,'specid'])
    plotspecinspect(matched_table,obj_index,filter=image_filter)     
    #break